<a href="https://colab.research.google.com/github/CagataySencan/Hybrid-Movie-Recommender-System/blob/main/HybridMovieRecommanderEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genel Hazırlıklar

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
!pip install scikit-surprise
from surprise import Reader,Dataset,SVD

In [ ]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download rounakbanik/the-movies-dataset
! kaggle datasets download tmdb/tmdb-movie-metadata

In [ ]:
! unzip /content/the-movies-dataset.zip
! unzip /content/tmdb-movie-metadata.zip

In [ ]:
credits = pd.DataFrame(pd.read_csv('credits.csv'))
keywords = pd.DataFrame(pd.read_csv('keywords.csv'))
movies_metadata_tmdb = pd.DataFrame(pd.read_csv('tmdb_5000_movies.csv'))
movies_tmdb_credits = pd.DataFrame(pd.read_csv('tmdb_5000_credits.csv'))
movies_metadata = pd.DataFrame(pd.read_csv('movies_metadata.csv'))
ratings = pd.DataFrame(pd.read_csv('ratings.csv'))

# Content Based Filtering


In [ ]:
# Content based filtering için hazırlık

content_based_movies = movies_metadata_tmdb[['overview','title','genres', 'keywords']]
content_based_credits = movies_tmdb_credits[['cast','crew','movie_id']]
content_based = content_based_movies.join(content_based_credits)
# Yeterli veri olduğu için eksik veri olan satırları çıkartma kararı aldım
content_based = content_based.dropna()

# Bu filtreden en yüksek verimi alabilmek için iki alt filtreye bölme kararı aldım
# İlk filtre önerileri overview'a, ikincisi ise cast, crew, keyword, genre gibi parametrelere göre öneri yapacak
# Bu filtrede memory yetersizliği nedeniyle cosine similarity score hesaplayamadığım için TMDB 5000 Movie Dataset'i kullandım
content_based.head()

### Overview Bazlı Filtre

In [ ]:
# overview bazlı filtre :

overview_based = content_based[['title','overview']]
print(overview_based.isnull().sum())
print(overview_based.head())

# Vektörleştirme işlemi 

tf_idf = TfidfVectorizer(stop_words='english')
tfidf_matrix_overview = tf_idf.fit_transform(overview_based['overview'])


# Filmler arasındaki benzerlikleri nümerik bir skorla görmek için 'cosine similarity score' hesabı yapma kararı aldım
tfidf_matrix_overview.shape
similarity_overview = linear_kernel(tfidf_matrix_overview,tfidf_matrix_overview)
overview_based = overview_based.reset_index()
index = pd.Series(overview_based.index, index=overview_based['title']).drop_duplicates()

In [11]:
# Girilen film ismine göre öneri yapacak olan fonksiyon :

def recommend_by_overview(title, sim = similarity_overview) :
  idx = index[title]
  sim_score = list(enumerate(sim[idx]))
  sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)
  sim_score = sim_score[1:11]
  movie_index = [i[0] for i in sim_score]
  recommendation = overview_based['title'].iloc[movie_index].tolist()
  recDict = {}
  i = 0
  while i < 10 :
    recDict[recommendation[i]] = sim_score[i][1]
    i += 1

  return recDict

### Cast, Crew, Keyword, Genre Bazlı Filtre

In [ ]:
features = ['cast', 'crew', 'keywords', 'genres']
feature_based = content_based[['title','cast', 'crew', 'keywords', 'genres']]

for feature in features : 
  feature_based[feature] = feature_based[feature].apply(literal_eval)

# Yönetmenleri crew sütunundan almak için gerekli fonksiyon  
def add_director(j) :
  for i in j :
    if i['job'] == 'Director':
      return i['name']
  return np.nan

# Diğer sütunlardaki bilgilerden ilk 5 tanesini almak için gerekli fonksiyon
def add_list(j) :
  if isinstance(j, list):
        
        names = [i['name'] for i in j]
        
        if len(names) > 5:
            names = names[:5]
        return names
   
  return []       

# Veriyi kullanabilmek için uyumlu forma getirme işlemi
feature_based['director'] = feature_based['crew'].apply(add_director)
features = ['cast', 'keywords', 'genres']

for feature in features:
    feature_based[feature] = feature_based[feature].apply(add_list)

feature_based = feature_based.drop(columns = ['crew'])

In [13]:
# Bütun satırlardaki verileri küçük harfe dönüştürme ve boşlukları düzenleme işlemi 
def clean(j):
    if isinstance(j, list):
        return [str.lower(i.replace(" ", "")) for i in j]
    else:
        if isinstance(j, str):
            return str.lower(j.replace(" ", ""))
        else:
            return ''

features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    feature_based[feature] = feature_based[feature].apply(clean)

# Vektörleştirme işlemini tek seferde yapabilmek için bütün verileri içeren tek bir sütun oluşturma işlemi
def all_data(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


feature_based['all'] = feature_based.apply(all_data, axis=1)

tf_idf = TfidfVectorizer(stop_words='english')
tfidf_matrix_feature = tf_idf.fit_transform(feature_based['all'])

# Filmler arasındaki benzerlikleri nümerik bir skorla görmek için 'cosine similarity score' hesabı yapma kararı aldım
tfidf_matrix_feature.shape
similarity_feature = linear_kernel(tfidf_matrix_feature,tfidf_matrix_feature)
feature_based = feature_based.reset_index()
index = pd.Series(feature_based.index, index=feature_based['title']).drop_duplicates()

In [14]:
def recommend_by_feature(title, sim = similarity_feature) :
  idx = index[title]
  sim_score = list(enumerate(sim[idx]))
  sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)
  sim_score = sim_score[1:11]
  movie_index = [i[0] for i in sim_score]
  recommendation = feature_based['title'].iloc[movie_index].tolist()
  recDict = {}
  i = 0
  while i < 10 :
    recDict[recommendation[i]] = sim_score[i][1]
    i += 1

  return recDict

In [ ]:
# İki filtreden de gelen veriler dictionary şeklinde olduğundan dolayı değerden anahtar kelimeyi bulan bir fonksiyona ihtiyacım oldu
def get_key(d, val):
    keys = [k for k, v in d.items() if v == val]
    if keys:
        return keys[0]
    return None

# Buradaki fonksiyonda iki filtreden gelen cosine similarity score değeri bir listeye atanıyor 
# Liste büyükten küçüğe sıralanıp değeri en büyük olan 10 film öneri olarak veriliyor
def final_recommendation_content_based(title) :
  list1 = recommend_by_overview(title)
  list2 = recommend_by_feature(title)
  value_list = list(list1.values()) + list(list2.values())
  value_list.sort(reverse = True)
  recommend_list = []
  i = 0 
  while i < 20 :
    key = get_key(list1,value_list[i])
    if key is None :
      new_key = get_key(list2,value_list[i])
      if new_key in recommend_list :
        i += 1 
        continue
      else :
        recommend_list.append(new_key)    
    else :
      if key in recommend_list :
        i += 1 
        continue
      else :
        recommend_list.append(key)  
    i += 1  
  recommend_list = recommend_list[0:10]
  return recommend_list

final_recommendation_content_based('Batman Begins')

# Collaborative Filtering